# 导入相应的库

In [ ]:
# 数据处理库
import numpy as np
import pandas as pd

# 数据分割库
from sklearn.model_selection import train_test_split

# 数据标准化库
from sklearn.preprocessing import StandardScaler

# 模型评价库
from sklearn.metrics import accuracy_score

# 深度学习库
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Flatten, BatchNormalization, Dropout, Activation
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam

# 数据读取

In [ ]:
df = pd.read_csv("drive/MyDrive/dataset/sample.csv")

In [ ]:
df = df.drop("Unnamed: 0", axis=1)

In [ ]:
df['Label1'].value_counts(normalize=True, sort=True)

# 训练集和测试集准备

In [ ]:
data = df.iloc[:, :-1]
target = df.iloc[:, -1]

In [ ]:
# 数据分割
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=0.3)

In [ ]:
# 数据标准化
std = StandardScaler()
x_train = std.fit_transform(x_train)
x_test = std.fit_transform(x_test)

In [ ]:
# 数据维度调整
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

# 训练模型

In [ ]:
def train():
    # 参数定义
    print("========================定义参数中========================")
    num_classes = 2
    learning_rate = 0.000005


    # 模型初始化
    print("========================模型初始化中========================")
    cnn_model = Sequential()
    cnn_model.add(Conv1D(32, 3, activation='relu', input_shape=(x_train.shape[1], 1)))
    cnn_model.add(BatchNormalization())
    cnn_model.add(Activation('relu'))
    cnn_model.add(MaxPooling1D(2))
    cnn_model.add(Dropout(0.1))
    cnn_model.add(Conv1D(64, 3, activation='relu'))
    cnn_model.add(BatchNormalization())
    cnn_model.add(Activation('relu'))
    cnn_model.add(MaxPooling1D(2))
    cnn_model.add(Dropout(0.1))
    cnn_model.add(Flatten())
    cnn_model.add(BatchNormalization())
    cnn_model.add(Dense(64, activation='relu'))
    cnn_model.add(Dropout(0.1))
    cnn_model.add(Dense(1, activation='sigmoid'))

    print("========================模型编译中========================")
    # 编译模型
    optimizer = Adam(learning_rate=learning_rate)
    cnn_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    print("========================模型训练中========================")
    # 训练模型
    cnn_model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

    print("========================模型验证中========================")
    # 验证模型
    validation_loss, validation_accuracy = cnn_model.evaluate(x_test, y_test)

    print("========================模型保存中========================")
    # 保存模型
    cnn_model.save('cnn_model.h5')

# 模型评价

In [ ]:
def test():
    cnn_model = load_model('cnn_model.h5')
    validation_loss, validation_accuracy = cnn_model.evaluate(x_test, y_test)
    y_pred_prob = cnn_model.predict(x_test)
    y_pred = np.where(y_pred_prob >= 0.5, 1, 0)
    y_pred = np.squeeze(y_pred)

    # 计算Accuracy
    accuracy = tf.keras.metrics.Accuracy()
    accuracy.update_state(y_test, y_pred)
    accuracy_result = accuracy.result()

    # 计算Precision
    precision = tf.keras.metrics.Precision()
    precision.update_state(y_test, y_pred)
    precision_result = precision.result()

    # 计算Recall
    recall = tf.keras.metrics.Recall()
    recall.update_state(y_test, y_pred)
    recall_result = recall.result()

    # 计算F1 Score
    f1_score = 2 * (precision_result * recall_result) / (precision_result + recall_result)

    print(f'Loss：{validation_loss}')
    print(f'Accuracy：{validation_accuracy}')
    print("Precision:", precision_result)
    print("Recall:", recall_result)
    print("F1 Score:", f1_score)
    return validation_accuracy

In [ ]:
train()

In [ ]:
test()

In [ ]:
acc